[빠른 시작(QUICKSTART)](https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html#id3)

## 데이터 작업하기

In [ ]:
"""
torch.utils.data.Dataset : 샘플과 정답 저장
torch.utils.data.DataLoader : Dataset을 순회 가능한 객체로 감싸는 것
"""

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
"""
PyTorch가 제공하는 도메인 특화 라이브러리 : TorchText, TorchVision, TorchAudio
    - 해당 라이브러리들은 데이터셋을 함께 제공
    - torchvision.dataset : CIFAR, COCO 같은 데이터셋을 포함
    - 모든 torchvision.dataset은 샘플/정답을 변경하기 위한 transform/target_transform을 포함
"""

# 학습 데이터 다운로드
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
# 테스트 데이터 다운로드
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 15693005.10it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 274930.50it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4990982.83it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3999310.43it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
"""
DataLoader의 인자로 Dataset을 전달
    - Dataset을 순회 가능한 객체로 감싸기
        -> 자동화된 배치, 샘플링, 셔플 및 다중 프로세스로 데이터 불러오는 것 지원
"""
batch_size = 64

# 데이터로더 생성
train_dataloader = DataLoader(
    training_data,
    batch_size=batch_size,
)
test_dataloader = DataLoader(
    test_data,
    batch_size=batch_size,
)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 모델 만들기

In [ ]:
"""
nn.Module을 상속받는 클래스를 정의하여 신경망 모델을 생성
    - __init__ 함수에서 신경망의 계층을 정의
    - forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정
    - 가능한 경우 GPU/MPS로 연산을 가속
"""

# 장치 설정
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 매개변수 최적화하기

In [ ]:
"""
모델을 학습하려면
    1. Loss function
    2. Optimizer
"""

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
"""
각 학습 단계에서 모델은 학습 데이터셋에 대한 예측 수행,
예측 오류를 역전파하며 모델의 매개변수 조정
"""
def train(
    dataloader,
    model,
    loss_fn,
    optimizer,
):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파 수행
        # 미분값이 누적되는 것을 방지하기 위해 .zero_grad()
        optimizer.zero_grad()

        # require_grad=True인 모든 텐서들에 대해 .backward()로 gradient 계산
        # loss 함수에서부터 계산을 시작하므로 loss 에 .backward()를 적용해준다
        loss.backward()

        # 파라미터 업데이트
        optimizer.step()

        if batch % 100 == 0:
            # .item() : 손실이 가지고 있는 스칼라 값을 얻기
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [ ]:
# 모델의 성능을 확인하기 위해 테스트 데이터셋으로 모델의 성능 확인
def test(
    dataloader,
    model,
    loss_fn,
):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    # 모델을 evaluation 모드로 변경해주기 (dropout 같은 것 off 등...)
    # .train(False) 와 동일
    model.eval()

    test_loss, correct = 0, 0

    # autograd engine 비활성화 시켜 필요한 메모리 줄여주고, 연산속도 증가시키기
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
"""
학습 단계는 여러번의 반복(epoch)을 거쳐서 수행
각 에폭마다 모델의 정확도 및 손실을 출력
"""

epochs = 5
for t in range(epochs):
    print(f"Epoch: {t+1}\n---------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch: 1
---------------------------
loss: 2.301769 [   64/60000]
loss: 2.293076 [ 6464/60000]
loss: 2.275995 [12864/60000]
loss: 2.265576 [19264/60000]
loss: 2.252063 [25664/60000]
loss: 2.230810 [32064/60000]
loss: 2.238633 [38464/60000]
loss: 2.209691 [44864/60000]
loss: 2.194334 [51264/60000]
loss: 2.170866 [57664/60000]
Test Error: 
 Accuracy: 37.3%, Avg loss: 2.162173 

Epoch: 2
---------------------------
loss: 2.175721 [   64/60000]
loss: 2.167448 [ 6464/60000]
loss: 2.111089 [12864/60000]
loss: 2.117507 [19264/60000]
loss: 2.078017 [25664/60000]
loss: 2.023846 [32064/60000]
loss: 2.052016 [38464/60000]
loss: 1.979047 [44864/60000]
loss: 1.966791 [51264/60000]
loss: 1.915298 [57664/60000]
Test Error: 
 Accuracy: 53.9%, Avg loss: 1.902573 

Epoch: 3
---------------------------
loss: 1.936883 [   64/60000]
loss: 1.912104 [ 6464/60000]
loss: 1.790183 [12864/60000]
loss: 1.820109 [19264/60000]
loss: 1.725746 [25664/60000]
loss: 1.680517 [32064/60000]
loss: 1.701615 [38464/60000]
lo

## 모델 저장하기

In [ ]:
"""
모델 저장의 일반적인 방법
    내부 상태 사전을 직렬화 하는 것
    -> serialize internal state dictionary
"""

# torch.save() 객체를 디스크에 저장, pickle 모듈을 이용하여 객체를 직렬화
# 모든 종류의 모델 및 tensor를 저장하는 것이 가능
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기

In [ ]:
# 모델 구조를 다시 만든 뒤 상태 사전(state_dict)을 모델에 불러오기
model = NeuralNetwork().to(device)

# torch.load: pickle 모듈 이용, 객체를 역직렬화하여 메모리에 할당
# load_state_dict : state_dict를 사용하여 모델의 매개변수 불러오기
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
# 모델을 사용해서 예측 수행하기
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: 'Ankle boot', Actual: 'Ankle boot'
